In [ ]:
#javascript for console of Colab
# function ClickConnect(){
# 	console.log("Working"); 
# 	document.querySelector("colab-toolbar-button").click() 
# } setInterval(ClickConnect, 1800000)

# import Library

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
from collections import OrderedDict #for multiGPU
from tqdm import tqdm

import os
import random
import time

# GPU

In [2]:
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
#for colab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is ", device)

device is  cuda


# Load Data

In [3]:
print("loading dataset")

data_transforms = {
    'train' : transforms.Compose([ #horizontal flip, random crop, normalization           
            #transforms.ToPILImage(),
            # transforms.Resize((256,256)),
            transforms.RandomCrop(32, padding = 4), #augmentation, input for ResNet
            transforms.RandomHorizontalFlip(), #augmentation
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), #with the per-pixel mean subtracted
            #normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     #std=[0.229, 0.224, 0.225])

        ]) , 
        'test': transforms.Compose([
            transforms.ToTensor(), #single view of the original 32x32 image
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    }
#access data_transforms['train']

#dataloader (batch)
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform = data_transforms['train']
)
trainloader = torch.utils.data.DataLoader(
    train_set, batch_size = 128, shuffle = True, num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)

#classes
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')


loading dataset


100%|██████████| 170498071/170498071 [00:02<00:00, 70260544.12it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
print(len(train_set))
print(len(test_set))

50000
10000


# epochs, iterations

In [ ]:
Iterations = 48000
total_epochs = int(Iterations/len(trainloader))

print(total_epochs)

122


# Model Architecture

In [ ]:
# x = torch.zeros(1,16,32,32)
# k = nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False)
# y = k(x)
# print(y.shape)

torch.Size([1, 16, 32, 32])


In [ ]:
# x = torch.zeros(1,16,32,32)
# k = nn.Conv2d(16, 32, kernel_size = 3, stride = 2, padding = 1, bias = False)
# y = k(x)
# print(y.shape)

torch.Size([1, 32, 16, 16])


In [4]:
class optionA(nn.Module):
  def __init__(self, func):
    super(optionA, self).__init__()
    self.func = func
  
  def forward(self, x):
    return self.func(x)

In [5]:
class Fx2_16(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16)
    )
    self.residual = nn.Sequential()
  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out +=self.residual(x)
    out = F.relu(out)

    return out

In [6]:
class Fx2_32(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
    )
    self.residual = nn.Sequential()
  
  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out += self.residual(x)
    out = F.relu(out)
    return out


In [7]:
class Fx2_64(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Sequential(
         nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.residual = nn.Sequential()

  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out += self.residual(x)
    out = F.relu(out)
    return out

In [8]:
#classes for Resnet20 Model
class conv16 (nn.Module): #Blue
  def __init__(self):
    super().__init__()
    self.Fx1 = Fx2_16()
    self.Fx2 = Fx2_16()
    self.Fx3 = Fx2_16()
    
  def forward(self,x):
    x1 = self.Fx1(x) 
    x2 = self.Fx2(x1)
    x3 = self.Fx3(x2) 
 
    return x3

#classes for Resnet20 Model
class conv32 (nn.Module): #Purple
  def __init__(self):
    super().__init__()
    self.Fx1_1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 32, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(32))
    self.Fx1_2 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32)
    )
    ##OptionB
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(16,32,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(32)
    # )

    ## OptionA
    self.residual_mapping = optionA(lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, 32//4, 32//4), "constant", 0))

    self.Fx2 = Fx2_32()
    self.Fx3 = Fx2_32()
    
  def forward(self,x):
    out = F.relu(self.Fx1_1(x))
    out = self.Fx1_2(out) 
    out += self.residual_mapping(x)
    x1 = F.relu(out)

    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2) 

    return x3

#classes for Resnet20 Model
class conv64 (nn.Module): #Green
  def __init__(self):
    super().__init__()
    self.Fx1_1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.Fx1_2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    ##OptionB
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(32,64,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(64)
    # )

    ## OptionA
    self.residual_mapping = optionA(lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, 64//4, 64//4), "constant", 0))
  
    self.Fx2 = Fx2_64()
    self.Fx3 = Fx2_64()
    
  def forward(self,x):
    out = F.relu(self.Fx1_1(x))
    out = self.Fx1_2(out)
    out += self.residual_mapping(x)
    x1 = F.relu(out)

    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2) 

    return x3


In [ ]:
# x = torch.zeros(1,16,32,32)
# k = conv16()
# y = k(x)
# print(y.shape)

torch.Size([1, 16, 32, 32])


In [ ]:
# x = torch.zeros(1,16,32,32)
# k = conv32()
# y = k(x)
# print(y.shape)

torch.Size([1, 16, 32, 32])
torch.Size([1, 32, 16, 16])
torch.Size([1, 32, 16, 16])
torch.Size([1, 32, 16, 16])


In [9]:
class MyModel(nn.Module): #ResNet20 architecture implementation
    def __init__(self, conv2 = conv16, conv3 = conv32,
            conv4 = conv64, num_classes = 10, init_weights = True): #num_classes
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding=1, bias = False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        self.conv2 = conv2()
        self.conv3 = conv3()
        self.conv4 = conv4()
        #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fclayer = nn.Linear(64, num_classes)

        if init_weights:
            self.initialize_weights()
    
    # define weight initialization function
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0) 
        
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x = F.avg_pool2d(x4, x4.size()[3])
        #x = self.avg_pool(x4)
        x = x.view(x.size(0), -1)
        output = self.fclayer(x)
        return output

In [ ]:
from torchsummary import summary
resnet20 = MyModel()
summary(resnet20, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
            Conv2d-6           [-1, 16, 32, 32]           2,304
       BatchNorm2d-7           [-1, 16, 32, 32]              32
            Fx2_16-8           [-1, 16, 32, 32]               0
            Conv2d-9           [-1, 16, 32, 32]           2,304
      BatchNorm2d-10           [-1, 16, 32, 32]              32
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
           Fx2_16-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,

In [ ]:
resnet20

MyModel(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): conv16(
    (Fx1): Fx2_16(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (residual): Sequential()
    )
    (Fx2): Fx2_16(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): Sequent

# Train

In [ ]:
# print("building model")
# net = MyModel()
# net = net.to(device)

building model


In [10]:
# #get_model when loading model parameters trained w/ multi-gpu
def get_model(model_name, checkpoint_path):
    print("start loading model parameter")
    model = model_name()
    state_dict = torch.load(checkpoint_path)

    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        name = k.replace('module.', '')
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    print("model parameter loaded")
    return model

In [11]:

def train(n_epochs, train_loader) :
 
    epochs = n_epochs
    train_dataloader = train_loader
    #get current time -> folder name
    train_date = time.strftime("%y%m%d_%H%M", time.localtime(time.time()))
    # print(train_date)

    #log path for tensorboard
    log_path = os.path.join(os.getcwd(),"logs",train_date)
    print("log_path is {}".format(log_path))

    #model path to save '.pth'file
    model_path = os.path.join(os.getcwd(),"models",train_date)
    print("model_path is {}".format(model_path))
    if os.path.isdir(model_path) == False:
            os.makedirs(model_path)

    #writer for tensorboard
    writer = SummaryWriter(log_path)
 
    # whether available GPU
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    #define model (처음부터 학습시)
    print("처음부터 학습")
    model = MyModel()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    ####### 이전 학습한거에 추가적으로 학습시
    # print("이전에 하던거 부터 학습")
    # model_name = MyModel
    # # /content/models/230412_2145/0.pth
    # checkpoint_path = './12.pth'  #수정 필요
    # model = get_model(model_name,checkpoint_path)
    # model.to(device)

    #######

    #define loss funcion, optimizer, (scheduler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=0.1, momentum=0.9,weight_decay=0.0001)
    #lr = 0.025, momentum = 0.9, weight_decay = 0.0005(original)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [81,122], gamma = 0.1)
    # 0.1 -> 0.01 (32K iterations = 81 epochs) -> 0.001 (48K iterations = 122 epochs)

    print("start training")
    #Train
    for epoch in range(epochs):
        
        model.train(True)
        len_trainbatch = len(train_dataloader)

        i = 0
        for inputs, targets in tqdm(train_dataloader):
            i = i+1
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            writer.add_scalar('train/loss', loss.item(), epoch*len_trainbatch+i) #iterations

        scheduler.step()    
        print("train loss of {}th epoch is {}".format(epoch,loss.item())) ## 수정 필요
 
        # Save every N(30) epoch
        save_epoch = 30
        if save_epoch > 0 and epoch % save_epoch == save_epoch-1:
            state_dict = model.state_dict()
            torch.save(state_dict, os.path.join(model_path, str(epoch+1)+'.pth')) #epoch+1로 바꾸기
        
    print('Finished Training')

    #save model after training
    torch.save(model.state_dict(), './model.pth') 

   

# Main

In [12]:

def main() :

    random.seed(53)
    train(163, trainloader) #64k iterations = 163 epochs

   

if __name__ == '__main__':
    main()

log_path is /content/logs/230417_0009
model_path is /content/models/230417_0009
처음부터 학습
start training


100%|██████████| 391/391 [00:36<00:00, 10.73it/s]


train loss of 0th epoch is 1.0763870477676392


100%|██████████| 391/391 [00:27<00:00, 14.32it/s]


train loss of 1th epoch is 0.9113261103630066


...

train loss of 160th epoch is 0.02748122252523899


100%|██████████| 391/391 [00:26<00:00, 15.03it/s]


train loss of 161th epoch is 0.02905499003827572


100%|██████████| 391/391 [00:26<00:00, 15.02it/s]

train loss of 162th epoch is 0.08574984967708588
Finished Training


# Test

- model.pth   

100 100 Loss: 0.343 | Acc: 91.560% (9156/10000)


- 30.pth

100 100 Loss: 0.468 | Acc: 84.730% (8473/10000)


- 60.pth


100 100 Loss: 0.438 | Acc: 86.170% (8617/10000)

- 90.pth


100 100 Loss: 0.279 | Acc: 91.710% (9171/10000)

- 120.pth

100 100 Loss: 0.353 | Acc: 91.210% (9121/10000)


- 150.pth


100 Loss: 0.342 | Acc: 91.450% (9145/10000)


In [23]:
def test():

    ####### not touch
    model_name = MyModel
    # /content/models/230412_2145/0.pth
    #/content/models/230412_2208/1.pth
    #/content/models/230416_0927/90.pth
    #/content/models/230417_0009/30.pth
    checkpoint_path = './models/230417_0009/150.pth'  #수정 필요
    mode = 'test' 
#     data_dir = "./test_data"
#     meta_path = "./answer.json"
    model = get_model(model_name,checkpoint_path)
    #######
    
#     # Create training and validation datasets
#     test_datasets = MyDataset(meta_path, data_dir, data_transforms['test'],is_train= False) #self,meta_path,root_dir,transform=None, is_train = True

#     # Create training and validation dataloaders
#     test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False, num_workers=4)

    test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
    
    testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)
    
    # Detect if we have a GPU available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    # Send the model to GPU
    #multi-GPU
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)

    # Set model as evaluation mode
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    
    # Inference
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        batch_idx = 0
        for inputs, targets in tqdm(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_idx = batch_idx + 1
            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))



# Main

In [24]:

def main() :

    random.seed(53)
    test()

   
if __name__ == '__main__':
    main()

start loading model parameter
model parameter loaded
Files already downloaded and verified


  5%|▌         | 5/100 [00:00<00:05, 17.61it/s]

1 100 Loss: 0.141 | Acc: 96.000% (96/100)
2 100 Loss: 0.198 | Acc: 93.500% (187/200)
3 100 Loss: 0.236 | Acc: 92.667% (278/300)
4 100 Loss: 0.239 | Acc: 92.750% (371/400)
5 100 Loss: 0.231 | Acc: 93.600% (468/500)
6 100 Loss: 0.218 | Acc: 94.167% (565/600)


 12%|█▏        | 12/100 [00:00<00:03, 26.75it/s]

7 100 Loss: 0.223 | Acc: 94.143% (659/700)
8 100 Loss: 0.248 | Acc: 93.500% (748/800)
9 100 Loss: 0.273 | Acc: 93.000% (837/900)
10 100 Loss: 0.282 | Acc: 92.600% (926/1000)
11 100 Loss: 0.280 | Acc: 92.455% (1017/1100)
12 100 Loss: 0.295 | Acc: 92.000% (1104/1200)
13 100 Loss: 0.294 | Acc: 91.769% (1193/1300)


...

86 100 Loss: 0.353 | Acc: 91.256% (7848/8600)
87 100 Loss: 0.351 | Acc: 91.287% (7942/8700)
88 100 Loss: 0.352 | Acc: 91.273% (8032/8800)
89 100 Loss: 0.352 | Acc: 91.270% (8123/8900)
90 100 Loss: 0.351 | Acc: 91.289% (8216/9000)
91 100 Loss: 0.349 | Acc: 91.330% (8311/9100)
92 100 Loss: 0.349 | Acc: 91.315% (8401/9200)


 97%|█████████▋| 97/100 [00:03<00:00, 31.17it/s]

93 100 Loss: 0.349 | Acc: 91.301% (8491/9300)
94 100 Loss: 0.348 | Acc: 91.319% (8584/9400)
95 100 Loss: 0.345 | Acc: 91.358% (8679/9500)
96 100 Loss: 0.345 | Acc: 91.365% (8771/9600)
97 100 Loss: 0.343 | Acc: 91.423% (8868/9700)
98 100 Loss: 0.342 | Acc: 91.459% (8963/9800)
99 100 Loss: 0.343 | Acc: 91.444% (9053/9900)
100

100%|██████████| 100/100 [00:03<00:00, 28.33it/s]

 100 Loss: 0.342 | Acc: 91.450% (9145/10000)


In [ ]:
def test():

    ####### not touch
    model_name = MyModel
    # /content/models/230412_2145/0.pth
    #/content/models/230412_2208/1.pth
    #/content/models/230415_0435/160.pth
    checkpoint_path = './models/230415_0435/150.pth'  #수정 필요
    mode = 'test' 
#     data_dir = "./test_data"
#     meta_path = "./answer.json"
    model = get_model(model_name,checkpoint_path)
    #######
    
#     # Create training and validation datasets
#     test_datasets = MyDataset(meta_path, data_dir, data_transforms['test'],is_train= False) #self,meta_path,root_dir,transform=None, is_train = True

#     # Create training and validation dataloaders
#     test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False, num_workers=4)

    test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
    
    testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)
    
    # Detect if we have a GPU available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    # Send the model to GPU
    #multi-GPU
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)

    # Set model as evaluation mode
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    
    # Inference
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        batch_idx = 0
        for inputs, targets in tqdm(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_idx = batch_idx + 1
            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))



In [ ]:

def main() :

    random.seed(53)
    test()

   
if __name__ == '__main__':
    main()

start loading model parameter
model parameter loaded
Files already downloaded and verified


  6%|▌         | 6/100 [00:00<00:03, 25.81it/s]

1 100 Loss: 0.493 | Acc: 75.000% (75/100)
2 100 Loss: 0.714 | Acc: 71.500% (143/200)
3 100 Loss: 0.808 | Acc: 71.333% (214/300)
4 100 Loss: 0.836 | Acc: 72.250% (289/400)
5 100 Loss: 0.862 | Acc: 71.600% (358/500)
6 100 Loss: 0.835 | Acc: 71.667% (430/600)
7 100 Loss: 0.871 | Acc: 71.143% (498/700)
8 100 Loss: 0.907 | Acc: 71.000% (568/800)
9 100 Loss: 0.948 | Acc: 70.333% (633/900)
10 100 Loss: 0.974 | Acc: 70.400% (704/1000)


 17%|█▋        | 17/100 [00:00<00:01, 41.56it/s]

11 100 Loss: 0.965 | Acc: 70.909% (780/1100)
12 100 Loss: 0.986 | Acc: 70.667% (848/1200)
13 100 Loss: 0.981 | Acc: 70.846% (921/1300)
14 100 Loss: 0.994 | Acc: 70.000% (980/1400)
15 100 Loss: 1.018 | Acc: 69.533% (1043/1500)
16 100 Loss: 1.034 | Acc: 69.000% (1104/1600)
17 100 Loss: 1.038 | Acc: 69.353% (1179/1700)
18 100 Loss: 1.056 | Acc: 69.000% (1242/1800)
19 100 Loss: 1.061 | Acc: 68.684% (1305/1900)
20 100 Loss: 1.076 | Acc: 68.550% (1371/2000)
21 100 Loss: 1.081 | Acc: 68.143% (1431/2100)


...

77 100 Loss: 1.106 | Acc: 67.870% (5226/7700)
78 100 Loss: 1.108 | Acc: 67.910% (5297/7800)
79 100 Loss: 1.108 | Acc: 67.975% (5370/7900)
80 100 Loss: 1.110 | Acc: 67.963% (5437/8000)
81 100 Loss: 1.108 | Acc: 67.963% (5505/8100)
82 100 Loss: 1.105 | Acc: 67.976% (5574/8200)
83 100 Loss: 1.102 | Acc: 68.060% (5649/8300)
84 100 Loss: 1.104 | Acc: 68.024% (5714/8400)
85 100 Loss: 1.107 | Acc: 67.988% (5779/8500)
86 100 Loss: 1.109 | Acc: 67.942% (5843/8600)
87

 97%|█████████▋| 97/100 [00:02<00:00, 51.78it/s]

 100 Loss: 1.108 | Acc: 67.989% (5915/8700)
88 100 Loss: 1.106 | Acc: 68.057% (5989/8800)
89 100 Loss: 1.107 | Acc: 67.989% (6051/8900)
90 100 Loss: 1.106 | Acc: 67.956% (6116/9000)
91 100 Loss: 1.105 | Acc: 67.901% (6179/9100)
92 100 Loss: 1.104 | Acc: 67.870% (6244/9200)
93 100 Loss: 1.107 | Acc: 67.806% (6306/9300)
94 100 Loss: 1.107 | Acc: 67.777% (6371/9400)
95 100 Loss: 1.107 | Acc: 67.811% (6442/9500)
96 100 Loss: 1.105 | Acc: 67.844% (6513/9600)
97 100 Loss: 1.103 | Acc: 67.876% (6584/9700)


100%|██████████| 100/100 [00:02<00:00, 45.97it/s]

98 100 Loss: 1.105 | Acc: 67.857% (6650/9800)
99 100 Loss: 1.105 | Acc: 67.828% (6715/9900)
100 100 Loss: 1.105 | Acc: 67.850% (6785/10000)
